# Online retail clustering analysis

In [1]:
import pandas as pd

# Load the dataset into a pandas DataFrame
# Source: Chen,Daqing. (2015). Online Retail. UCI Machine Learning Repository. https://doi.org/10.24432/C5BW33
url = 'https://github.com/kjellrkri/Portfolio/raw/main/Online%20Retail/OnlineRetail.xlsx'

# Specify the engine as 'openpyxl' since the file is in .xlsx format
df = pd.read_excel(url, engine='openpyxl')

# Check the number of rows and columns in the DataFrame
num_rows, num_columns = df.shape
print(f'The DataFrame has {num_rows} rows and {num_columns} columns.')

# Display the first few rows of the DataFrame
df.head()

The DataFrame has 541909 rows and 8 columns.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Exploratory Data Analysis
### Dropp redundant columns

In [2]:
# Drop the redundant columns "InvoiceNo" and "StockCode"
df = df.drop(columns=['InvoiceNo', 'StockCode'])

# Display the DataFrame after dropping the redundant columns
print("\nDataFrame after dropping the redundant columns:")
df.head()


DataFrame after dropping the redundant columns:


,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Handle missing values

In [3]:
# Check for missing values in the entire DataFrame
missing_values = df.isnull().sum()

# Display the count of missing values in each column
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


135080 of the 541909, or about 25% of the rows contain missing values in the column CustomerID. Although this is a substantial amount we can not interpolate new IDs which would scew the results. Dropping the missing rows is therefore reasonable.

In [4]:
# Drop missing values from the DataFrame (overwrite the original DataFrame)
df.dropna(inplace=True)

# Display the DataFrame after dropping missing values
print("\nDataFrame after dropping missing values:")
df.head()


DataFrame after dropping missing values:


,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Handle datatypes

In [5]:
# Display the data types of the columns
print("Data types of the columns:")
df.dtypes

Data types of the columns:


Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [6]:
# Convert "Description" and "Country" columns to string data type (str)
df['Description'] = df['Description'].astype('string')
df['Country'] = df['Country'].astype('string')

# Convert "CustomerID" column to integer data type (int) for memory efficiency
df['CustomerID'] = pd.to_numeric(df['CustomerID'], errors='coerce', downcast='integer')

# Display the DataFrame after converting data types
print("\nDataFrame after converting data types:")
print(df.dtypes)


DataFrame after converting data types:
Description            string
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int16
Country                string
dtype: object
